In [1]:
import pandas as pd
import numpy as np

# Prepare Labeling
We're going to get ~2500 data with ~1250 of organization and the replies.  
For the organization data we're going to get the most replied and every tweet by organization we're going to search the most liked reply.

In [6]:
data_csv = ["CNNIndonesia.csv", "detikcom.csv", "tvOneNews.csv", "VIVAcoid.csv"]
NUM_ORGANIZTION = len(data_csv)
relative_path_data = "../../data/2. Filtered/"
organization_path = f"{relative_path_data}/organization"
replies_path = f"{relative_path_data}/replies"

organizaton_df = {data: pd.read_csv(f"{organization_path}/{data}")\
                          .sort_values("reply_count", ascending=False).reset_index(drop=True)
                 for data in data_csv}
replies_df = {data: pd.read_csv(f"{replies_path}/{data}") \
                      .sort_values("tweet", ascending=False)\
                      .set_index("tweet_conversation_id", drop=False)
              for data in data_csv}

NUM_TWEET_ORGANIZATION = int(np.ceil(1250 / NUM_ORGANIZTION)) 
print(f"We're going to get {NUM_TWEET_ORGANIZATION} tweets for every organization and reply")

We're going to get 313 tweets for every organization and reply


In [15]:
# Build Labeling ata
MIN_CHAR = 100

organization_labeling_df = {}
reply_labeling_df = {} 
for data_name in data_csv:
    current_organization = organizaton_df[data_name].copy()
    current_replies = replies_df[data_name].copy()
    
    # Get the headline data
    current_organization = current_organization[:NUM_TWEET_ORGANIZATION]
    
    # Get the reply data by matching tweet_id and conversation_id
    conversation_ids = current_organization['tweet_id'].values
    replies = []
    for conversation_id in conversation_ids:
        all_replies = current_replies.loc[conversation_id]
        
        # Only got one reply
        if isinstance(all_replies, pd.Series):
            replies.append(all_replies)
            continue
        
        # Got multiple reply
        if isinstance(all_replies, pd.DataFrame):
            appended_reply = all_replies.iloc[0]
            
            min_char_met = False
            for i in range(len(all_replies)):
                current_reply = all_replies.iloc[i]
                if len(current_reply["tweet"]) > MIN_CHAR:
                    appended_reply = current_reply
                    min_char_met = True
                    break
            
            if not min_char_met:
                print(f"Conversation id {conversation_id} has no met min char reply")
                    
            replies.append(all_replies.iloc[0])
            continue
        
        print(f"No Reply detected on conversation_id {conversation_id}")
        
    organization_labeling_df[data_name] = current_organization.copy()
    reply_labeling_df[data_name] = pd.DataFrame(replies).reset_index(drop=True)

Conversation id 1581210663229456384 has no met min char reply
Conversation id 1580058499467603968 has no met min char reply
Conversation id 1584812873498431488 has no met min char reply
Conversation id 1596357377241980928 has no met min char reply
Conversation id 1585932086648741890 has no met min char reply
Conversation id 1600602659865694233 has no met min char reply
Conversation id 1588405114661896192 has no met min char reply
Conversation id 1584400128521416706 has no met min char reply
Conversation id 1608658427949076481 has no met min char reply
Conversation id 1582051753310138370 has no met min char reply
Conversation id 1576474850713276416 has no met min char reply
Conversation id 1594570082688307200 has no met min char reply
Conversation id 1573728607855583232 has no met min char reply
Conversation id 1577226192608198656 has no met min char reply
Conversation id 1587611213454008322 has no met min char reply
Conversation id 1493129644979097602 has no met min char reply
Conversa

In [16]:
# Save data

relative_path_data_labeling = "../../data/3. Labeling Ready/"
organization_labeling_path = f"{relative_path_data_labeling}/organization/"
replies_labeling_path = f"{relative_path_data_labeling}/replies/"
    
# Combine for labeling
df_organization = []
df_replies = []
for i in organization_labeling_df:
    print(f"org {len(organization_labeling_df[i])} rep {len(reply_labeling_df[i])}")
    df_organization.append(organization_labeling_df[i])
    df_replies.append(reply_labeling_df[i])

concat_org = pd.concat(df_organization)  
# concat_org.to_csv(f"{relative_path_data_labeling}/organization.csv", index=False)
          
concat_rep =  pd.concat(df_replies)                 
# concat_rep.to_csv(f"{relative_path_data_labeling}/replies.csv", index=False)

with pd.ExcelWriter(f"{relative_path_data_labeling}/organization.xlsx") as writer:
  concat_org.to_excel(writer, sheet_name="Organization Sheet", index=False)

with pd.ExcelWriter(f"{relative_path_data_labeling}/replies_{MIN_CHAR}.xlsx") as writer:
  concat_rep.to_excel(writer, sheet_name="Replies Sheet", index=False)

org 313 rep 313
org 313 rep 313
org 313 rep 313
org 313 rep 313
